In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/random-user/sample_user.csv
/kaggle/input/random-user/__results__.html
/kaggle/input/random-user/__notebook__.ipynb
/kaggle/input/random-user/__output__.json
/kaggle/input/random-user/custom.css
/kaggle/input/dataset-version-3/dataver3.h5
/kaggle/input/dataver3/dataver3.h5
/kaggle/input/dataver3/__results__.html
/kaggle/input/dataver3/__notebook__.ipynb
/kaggle/input/dataver3/__output__.json
/kaggle/input/dataver3/custom.css
/kaggle/input/dataver2-train-test/__results__.html
/kaggle/input/dataver2-train-test/data_ver2.h5
/kaggle/input/dataver2-train-test/__notebook__.ipynb
/kaggle/input/dataver2-train-test/__output__.json
/kaggle/input/dataver2-train-test/custom.css
/kaggle/input/combine-dataset/__results__.html
/kaggle/input/combine-dataset/combine_data.h5
/kaggle/input/combine-dataset/__notebook__.ipynb
/kaggle/input/combine-dataset/__output__.json
/kaggle/input/combine-dataset/custom.css
/kaggle/input/dataver2/__results__.html
/kaggle/input/dataver2/data_ver2.h5
/kaggl

In [2]:
i=0
path=[
    "/kaggle/input/combine-dataset/combine_data.h5",
    "/kaggle/input/dataver2-train-test/data_ver2.h5",
    "/kaggle/input/dataver3/dataver3.h5"
     ]

In [3]:
%%capture
! pip install tensorflow==2.12.0
! pip install -U LibRecommender

In [4]:
import numpy as np
import pandas as pd
from libreco.data import split_by_num_chrono, DatasetFeat
from libreco.algorithms import DeepFM 

item_data = pd.read_json("/kaggle/input/preprocessing-course-json/course.json", lines=True)
user_item_data = pd.read_csv("/kaggle/input/preprocessing-user-json/user-course.csv")
user_data = pd.read_csv("/kaggle/input/preprocessing-user-json/user-5-100.csv")

In [5]:
combine_data = pd.read_hdf(path[i], key='df')
combine_data

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,...,exercise_num_problem,doing_exercise_id,doing_num_correct,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,user_course_num_comment,user_course_num_replies
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,...,"[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...",NaN,NaN,"[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang...",0,0,0
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci...",0,0,0
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]",NaN,NaN,"[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci...",13,0,0
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",NaN,NaN,NaN,NaN,NaN,NaN,386,0,0
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 6, 8, 1, 19, 8, 11, 7, 8, 9, 9, 1, 16, 5, ...",NaN,NaN,"[K_1,4-加成_化学, K_1-溴戊烷_化学, K_1-丁醇_化学, K_1-丁烯_化学...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1,4-addition, 1-bromopentane, 1-butanol, 1-bu...","[chemical, chemical, chemical, chemical, chemi...",1876,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[2, 1, 1, 1, 1, 2, 1, 3, 1, 16, 1, 3, 3, 2, 1,...",NaN,NaN,"[K_0号寄存器_计算机科学与技术, K_0号元素_计算机科学与技术, K_10进制_计算机...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[register 0, element 0, decimal, register 1, 8...","[computer science and technology, computer sci...",339,0,0
7473120,U_34712115,C_676937,1603225930,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 2, 2, 3, 1, 2, 4, 5, 1, 1, 1, 1, 4, 1, 1, ...",NaN,NaN,"[K_abort_计算机科学与技术, K_area函数_计算机科学与技术, K_array对...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[abort, area function, array object, begin fun...","[computer science and technology, computer sci...",370,0,0
7473121,U_34712115,C_694136,1603226018,data mining: theory and algorithm,,the most interesting theory + the most useful ...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[4, 1, 7, 4, 6, 5, 4, 5, 4, 7, 5, 5, 3, 6, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ..

In [6]:
def filter_user_item_interactions(df, user_col='user_id', item_col='course_id', min_interactions=5):
    """
    Lọc bỏ các user và item có ít hơn `min_interactions` tương tác.

    Args:
        df (pd.DataFrame): DataFrame chứa tương tác user-item.
        user_col (str): Tên cột user.
        item_col (str): Tên cột item (course).
        min_interactions (int): Số tương tác tối thiểu để giữ lại.

    Returns:
        pd.DataFrame: DataFrame đã được lọc.
    """
    import pandas as pd

    while True:
        original_len = len(df)
        
        # Lọc user
        user_counts = df[user_col].value_counts()
        valid_users = user_counts[user_counts >= min_interactions].index
        df = df[df[user_col].isin(valid_users)]

        # Lọc item
        item_counts = df[item_col].value_counts()
        valid_items = item_counts[item_counts >= min_interactions].index
        df = df[df[item_col].isin(valid_items)]

        if len(df) == original_len:
            break  # không còn gì để lọc nữa

    return df.reset_index(drop=True)
# filter_user_item_interactions

In [7]:
combine_data.columns

Index(['user_id', 'course_id', 'enroll_time', 'course_name',
       'course_prerequisites', 'course_about', 'course_field', 'school_id',
       'school_name', 'school_about', 'school_motto', 'teacher_id',
       'teacher_name', 'teacher_about', 'teacher_job_title',
       'teacher_org_name', 'video_id', 'video_ccid', 'video_duration',
       'user_gender', 'watch_id', 'total_watch_time', 'actual_watch_time',
       'exercise_id', 'exercise_num_problem', 'doing_exercise_id',
       'doing_num_correct', 'concept_id', 'concept_prerequisite',
       'concept_name', 'concept_field', 'course_total_comments',
       'user_course_num_comment', 'user_course_num_replies'],
      dtype='object')

In [8]:
item_col_list=[
    'course_id','course_name','course_prerequisites', 'course_about',
    'school_id','teacher_id','video_duration','exercise_num_problem','course_total_comments']
if i!=1:
    item_col_list+=['course_field']

item_data = combine_data[item_col_list]
item_data=item_data.groupby('course_id').first().reset_index()

In [9]:
item_data

,course_id,course_name,course_prerequisites,course_about,school_id,teacher_id,video_duration,exercise_num_problem,course_total_comments,course_field
0,C_1017355,introduction to traditional chinese medicine e...,the main subjects are college students who hav...,traditional chinese medicine culture - the tre...,[S_33],"[T_7970, T_7971, T_5826, T_7978, T_7979]",None,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",39624,None
1,C_1017419,smart government talk,,this course combines specific cases of traffic...,[S_52],"[T_7849, T_16199, T_16200, T_16201, T_16202, T...",None,"[15, 15, 20, 15, 15, 15, 15, 16, 15]",609,None
2,C_1025064,principles of ideological and political education,,"hello, dear classmate! do you want to know how...",[S_13],"[T_3915, T_3911, T_5426, T_5427, T_5429]","[466.575, 402.325, 476.65, 1025.9, 755.075, 42...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",696,None
3,C_1025076,advanced roller skating tutorial,,the true meaning of experiencing roller skatin...,[S_72],[T_5394],None,"[1, 1, 15, 1]",2,None
4,C_1025079,scientific fitness method-silk-winding fitness...,none,silk wrapping skills are the chinese solution ...,[S_97],[T_7939],"[671.97, 925.17, 748.11, 644.25, 543.93, 546.3...","[5, 5, 5, 5, 5]",125,None
...,...,...,...,...,...,...,...,...,...,...
3068,C_949542,thinking innovation and creativity development,none,if appearance is a letter of recommendation fo...,[S_749],"[T_7389, T_7390, T_10280, T_10281, T_7392, T_7...","[784.77, 713.7, 578.07, 689.01, 624.45, 658.17...","[7, 8, 6, 7, 8, 10, 7, 8, 8, 11, 10, 6, 6, 5, ...",839,None
3069,C_955163,learn to make mini programs - practical chapte...,"computer basics, html, css, javascript, progra...",this course is a wechat mini program learning ...,[S_1],"[T_172, T_14907, T_14933, T_1286]","[515.893, 134.584, 2084.548, 1546.264, 38.356,...","[5, 1, 10, 10, 1, 30, 4]",305,None
3070,C_956128,interpretation of hot issues of portal hyperte...,,interpretation of hot issues of portal hyperte...,[S_144],"[T_7005, T_7006, T_7007]",None,"[15, 15, 15, 15, 15, 15, 15]",0,None
3071,C_956129,introduction to games and esports,none,the game industry has become a popular profess...,[S_492],"[T_4850, T_4852, T_4851, T_4854]","[463.65, 598.375, 354.68, 547.18, 289.23, 349....","[2, 3, 3, 8, 7, 6, 10, 9, 9, 6, 4, 1, 6, 7, 8,...",1597,None


In [10]:
def prepare_course_features(course_df):
    course_df = course_df.copy()

    # Đảm bảo các cột danh sách là thực sự danh sách
    list_columns = ['school_id', 'teacher_id', 'video_duration', 'exercise_num_problem']
    for col in list_columns:
        course_df[col] = course_df[col].apply(lambda x: x if isinstance(x, list) else [])

    # ===== Tính toán đặc trưng =====
    course_df['total_video_duration'] = course_df['video_duration'].apply(
        lambda x: np.nansum(x) if len(x) > 0 else 0
    )
    course_df['total_num_problem'] = course_df['exercise_num_problem'].apply(
        lambda x: np.nansum(x) if len(x) > 0 else 0
    )
    course_df['num_videos'] = course_df['video_duration'].apply(len)
    course_df['num_exercises'] = course_df['exercise_num_problem'].apply(len)
    course_df['num_teachers'] = course_df['teacher_id'].apply(len)
    course_df['num_schools'] = course_df['school_id'].apply(len)

    # ===== Tách school_id_1 và school_id_2 =====
    course_df['teacher_id'] = course_df['teacher_id'].apply(lambda x: x[0] if len(x) > 0 else None)
    course_df['teacher_id'] = course_df['teacher_id'].fillna('missing')
    course_df['school_id_1'] = course_df['school_id'].apply(lambda x: x[0] if len(x) > 0 else None)
    course_df['school_id_2'] = course_df['school_id'].apply(lambda x: x[1] if len(x) > 1 else None)
    course_df['school_id_1'] = course_df['school_id_1'].fillna('missing')
    course_df['school_id_2'] = course_df['school_id_2'].fillna('missing')
    # ===== Embed teacher_id list (tạm: dùng chỉ số trung bình để đại diện) =====
    # Nếu muốn dùng embedding sau này thì giữ lại list hoặc dùng nhiều cột teacher_id_1, _2,...
    if i==1:
        course_df['course_text'] = course_df[
            ['course_name', 'course_prerequisites', 'course_about']
        ].fillna('').apply(lambda x: ' '.join(x), axis=1)
    else:
        course_df['course_field'] = course_df['course_field'].apply(
            lambda x: ' '.join(x) if isinstance(x, list) else ''
            )
        course_df['course_text'] = course_df[
            ['course_name', 'course_prerequisites', 'course_about','course_field']
        ].fillna('').apply(lambda x: ' '.join(x), axis=1)
    # ===== Output =====
    course_features = course_df[[
        'course_id',
        'course_text',
        'total_video_duration',
        'total_num_problem',
        'num_videos',
        'num_exercises',
        'num_teachers',
        'teacher_id',
        'num_schools',
        'school_id_1',
        'school_id_2',
        'course_total_comments'
    ]]

    return course_features


In [11]:
item_data = prepare_course_features(item_data)

In [12]:
item_data.isnull().sum()

course_id                0
course_text              0
total_video_duration     0
total_num_problem        0
num_videos               0
num_exercises            0
num_teachers             0
teacher_id               0
num_schools              0
school_id_1              0
school_id_2              0
course_total_comments    0
dtype: int64

In [13]:
user_data = combine_data[['user_id','user_gender']].drop_duplicates().reset_index()

In [14]:
user_item_data=combine_data[['user_id','course_id','enroll_time']]

In [15]:
sp_user = pd.read_csv("/kaggle/input/random-user/sample_user.csv")
user_data = user_data[user_data['user_id'].isin(sp_user['user_id'])]

In [16]:
# user_data = user_data.sample(n=10000, random_state=42)

In [17]:
user_course_filtered = user_item_data[user_item_data["user_id"].isin(user_data["user_id"])]
user_course_filtered = user_course_filtered[user_course_filtered["course_id"].isin(item_data["course_id"])]
user_course_filtered

,user_id,course_id,enroll_time
46,U_54,C_682442,1570587469
47,U_54,C_682164,1573174143
48,U_54,C_1748240,1586941976
49,U_54,C_1794464,1591966731
50,U_54,C_697791,1592061292
...,...,...,...
7473113,U_34712103,C_1916220,1602473670
7473114,U_34712103,C_735296,1602590307
7473115,U_34712103,C_677093,1602594530
7473116,U_34712103,C_854843,1602709450


In [18]:
def split_data(data):
    data.rename(columns={"course_id": "item","user_id":"user","enroll_time":"time"}, inplace=True)
    train_data, test_data = split_by_num_chrono(data, test_size=1)
    return train_data,test_data

In [19]:
data = user_course_filtered.merge(user_data.rename(columns={"id":"user_id"}), on='user_id', how='inner')
data['label'] = 1

In [20]:
data=data.drop('index',axis=1)
data

,user_id,course_id,enroll_time,user_gender,label
0,U_54,C_682442,1570587469,2,1
1,U_54,C_682164,1573174143,2,1
2,U_54,C_1748240,1586941976,2,1
3,U_54,C_1794464,1591966731,2,1
4,U_54,C_697791,1592061292,2,1
...,...,...,...,...,...
347101,U_34712103,C_1916220,1602473670,1,1
347102,U_34712103,C_735296,1602590307,1,1
347103,U_34712103,C_677093,1602594530,1,1
347104,U_34712103,C_854843,1602709450,1,1


In [21]:
item_data

,course_id,course_text,total_video_duration,total_num_problem,num_videos,num_exercises,num_teachers,teacher_id,num_schools,school_id_1,school_id_2,course_total_comments
0,C_1017355,introduction to traditional chinese medicine e...,0.000,109,0,19,5,T_7970,1,S_33,missing,39624
1,C_1017419,smart government talk this course combines sp...,0.000,141,0,9,8,T_7849,1,S_52,missing,609
2,C_1025064,principles of ideological and political educat...,55868.953,140,88,14,5,T_3915,1,S_13,missing,696
3,C_1025076,advanced roller skating tutorial the true mea...,0.000,18,0,4,1,T_5394,1,S_72,missing,2
4,C_1025079,scientific fitness method-silk-winding fitness...,20714.190,25,28,5,1,T_7939,1,S_97,missing,125
...,...,...,...,...,...,...,...,...,...,...,...,...
3068,C_949542,thinking innovation and creativity development...,22184.031,247,33,31,10,T_7389,1,S_749,missing,839
3069,C_955163,learn to make mini programs - practical chapte...,56372.024,61,100,7,4,T_172,1,S_1,missing,305
3070,C_956128,interpretation of hot issues of portal hyperte...,0.000,105,0,7,3,T_7005,1,S_144,missing,0
3071,C_956129,introduction to games and esports none the gam...,34165.387,145,70,23,4,T_4850,1,S_492,missing,1597


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


# Vectorizer
vectorizer = TfidfVectorizer(            
            max_features=30,
            stop_words='english',
            ngram_range=(1, 2))
vectorized_data = vectorizer.fit_transform(item_data['course_text'])
feature_names = [f'course_text_{i+1}' for i in range(len(vectorizer.get_feature_names_out()))]

# Chuyển thành DataFrame
tfidf_df = pd.DataFrame(vectorized_data.toarray(), columns=feature_names)

# Gộp vào DataFrame gốc
item_vec = item_data.join(tfidf_df)
item_vec.columns

Index(['course_id', 'course_text', 'total_video_duration', 'total_num_problem',
       'num_videos', 'num_exercises', 'num_teachers', 'teacher_id',
       'num_schools', 'school_id_1', 'school_id_2', 'course_total_comments',
       'course_text_1', 'course_text_2', 'course_text_3', 'course_text_4',
       'course_text_5', 'course_text_6', 'course_text_7', 'course_text_8',
       'course_text_9', 'course_text_10', 'course_text_11', 'course_text_12',
       'course_text_13', 'course_text_14', 'course_text_15', 'course_text_16',
       'course_text_17', 'course_text_18', 'course_text_19', 'course_text_20',
       'course_text_21', 'course_text_22', 'course_text_23', 'course_text_24',
       'course_text_25', 'course_text_26', 'course_text_27', 'course_text_28',
       'course_text_29', 'course_text_30'],
      dtype='object')

In [23]:
item_cols = list(item_vec.columns.drop(['course_id', 'course_text']))

In [24]:
train, test = split_data(data)

In [25]:
train

,user,item,time,user_gender,label
80372,U_100090,C_707022,1583235350,0,1
87074,U_100090,C_676687,1583752556,0,1
99387,U_100090,C_697073,1584917895,0,1
99392,U_100090,C_948038,1584918145,0,1
99396,U_100090,C_680716,1584918197,0,1
...,...,...,...,...,...
342777,U_997506,C_696673,1606727242,1,1
17803,U_99772,C_697032,1577444562,2,1
19231,U_99772,C_676701,1577871421,2,1
78125,U_99772,C_682485,1583079803,2,1


In [26]:
test

,user,item,time,user_gender,label
286393,U_100090,C_1755921,1600725730,0,1
329448,U_1002476,C_682463,1604703719,1,1
324637,U_100281,C_948078,1604091834,0,1
250208,U_10033817,C_677049,1598352038,0,1
332831,U_10033948,C_1875518,1605192192,2,1
...,...,...,...,...,...
223327,U_9969645,C_1784322,1596479205,1,1
286159,U_99722,C_1944971,1600715669,0,1
305653,U_99746,C_735143,1602198978,0,1
342778,U_997506,C_680747,1606727332,1,1


In [27]:
train = train.merge(item_vec.rename(columns={"course_id":"item"}), on='item', how='inner')
test = test.merge(item_vec.rename(columns={"course_id":"item"}), on='item', how='inner')

In [28]:
item_cols = [x for x in item_cols if x not in ['school_id_2',
                                               'teacher_id',
                                               'school_id_1',
                                               'course_total_comments',
                                               'num_videos',
                                               'num_exercises',
                                               'num_school'
                                               'num_teachers',
                                               'total_video_duration',
                                               'total_num_problem',
                                              ]]

In [29]:
# specify complete columns information
sparse_col = ['user_gender',
              # 'school_id_1',
              # 'teacher_id'
             ]
dense_col = [f for f in item_cols if f not in ['school_id_1','teacher_id']]
user_col = ["user_gender"]
item_col = item_cols

train_data, data_info = DatasetFeat.build_trainset(train, user_col, item_col, sparse_col, dense_col,shuffle=False)
test_data = DatasetFeat.build_testset(test,shuffle=False)

In [30]:
print(data_info)

n_users: 24581, n_items: 2951, data density: 0.4446 %


In [31]:
import tensorflow as tf
tf.compat.v1.reset_default_graph() 

In [32]:
model = DeepFM(
        "ranking",
        data_info,
        embed_size=16,
        n_epochs=50,
        lr=1e-4,
        lr_decay=False,
        reg=None,
        batch_size=2048,
        num_neg=1,
        use_bn=False,
        dropout_rate=None,
        hidden_units=(128, 64, 32),
        tf_sess_config=None,
        sampler="random"
)
start_time = time.time()
model.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    shuffle=True,
    eval_data=test_data,
    metrics=["recall", "precision", "ndcg", "map"],
    eval_batch_size=8192,
    k=10,
    eval_user_num=8192,
)
end_time = time.time()  # Kết thúc đo thời gian

print(f"Thời gian thực hiện: {end_time - start_time:.6f} giây")

Training start time: 2025-06-09 13:37:28
total params: 550,920 | embedding params: 468,916 | network params: 82,004


train: 100%|██████████| 315/315 [00:08<00:00, 37.58it/s]


Epoch 1 elapsed: 8.387s
	 train_loss: 0.6764


eval_listwise: 100%|██████████| 4096/4096 [01:31<00:00, 44.79it/s]


	 eval recall@10: 0.0278
	 eval precision@10: 0.0028
	 eval ndcg@10: 0.0125
	 eval map@10: 0.0080


...

	 eval recall@10: 0.1118
	 eval precision@10: 0.0112
	 eval ndcg@10: 0.0572
	 eval map@10: 0.0408


train: 100%|██████████| 315/315 [00:08<00:00, 36.74it/s]


Epoch 50 elapsed: 8.577s
	 train_loss: 0.305


eval_listwise: 100%|██████████| 4096/4096 [01:24<00:00, 48.54it/s]


	 eval recall@10: 0.1162
	 eval precision@10: 0.0116
	 eval ndcg@10: 0.0579
	 eval map@10: 0.0405
Thời gian thực hiện: 4753.123451 giây


In [33]:
def recommend_courses(model, user_course , num_recommendations=5):
    recommendations = {}

    # Dự đoán cho tất cả người dùng trong test_user với thanh tiến trình
    for user_id in tqdm(user_course['user'].unique(), desc="Khuyến nghị cho người dùng", unit="user"):
        # Dự đoán cho tất cả các khóa học của người dùng (tắt chi tiết trong dự đoán)
        recommended_courses = model.recommend_user(user=user_id,n_rec = num_recommendations)
        recommended_courses = {key: value.tolist() if isinstance(value, np.ndarray) else value for key, value in recommended_courses.items()}
        # Lấy top k khóa học dựa trên dự đoán
        recommendations = recommendations | recommended_courses
    return recommendations

In [34]:
def recommend_courses_with_negative_sampling(model, 
                                             user_course_train_df, 
                                             user_course_test_df, 
                                             all_course_ids, 
                                             num_negatives=100):
    """
    Trả về recommendations dạng {user_id: [item_ids]} gồm 1 positive + N negative, sắp xếp theo điểm dự đoán.
    
    Params:
        model: object có hàm predict(user_id, item_id)
        user_course_train_df: DataFrame với cột ['user_id', 'course_id']
        user_course_test_df: DataFrame với cột ['user_id', 'course_id'], mỗi user chỉ 1 dòng
        all_course_ids: list hoặc Series chứa toàn bộ course_id
        num_negatives: số lượng mẫu âm
    
    Returns:
        recommendations: dict {user_id: list[item_id]}, sắp xếp theo điểm dự đoán
    """
    from tqdm import tqdm
    import numpy as np

    # Tiền xử lý: gom user -> list[course_id]
    user_train_dict = user_course_train_df.groupby("user")["item"].apply(set).to_dict()
    user_test_dict = user_course_test_df.set_index("user")["item"].to_dict()
    
    recommendations = {}
    
    for i, (user_id, true_item) in enumerate(tqdm(user_test_dict.items(), 
                                                   desc="Khuyến nghị với negative sampling", unit="user")):

        train_items = user_train_dict.get(user_id, set())
        excluded_items = train_items.union({true_item})

        # Lấy negative items chưa tương tác
        negative_candidates = [item for item in all_course_ids if item not in excluded_items]

        if len(negative_candidates) < num_negatives:
            continue  # Bỏ qua user nếu không đủ negative samples

        negative_samples = np.random.choice(negative_candidates, num_negatives, replace=False).tolist()
        candidate_items = [true_item] + negative_samples

        # Dự đoán điểm
        user_ids = [user_id] * len(candidate_items)
        scores = model.predict(user_ids, candidate_items)
        ranked_items = [item for _, item in sorted(zip(scores, candidate_items), reverse=True)]

        recommendations[user_id] = ranked_items

    return recommendations


In [35]:
import numpy as np

# Precision@5
def precision_at_k(y_true, recommendations, k=5):
    precision_scores = []
    for user, recommended_courses in recommendations.items():
        true_courses = y_true.get(user, [])
        if not true_courses:
            continue
        top_k_pred = recommended_courses[:k]
        relevant = sum(1 for course in top_k_pred if course in true_courses)
        precision_scores.append(relevant / k)
    return np.mean(precision_scores) if precision_scores else 0.0

# Recall@5
def recall_at_k(y_true, recommendations, k=5):
    recall_scores = []
    for user, recommended_courses in recommendations.items():
        true_courses = y_true.get(user, [])
        if not true_courses:
            continue
        top_k_pred = recommended_courses[:k]
        relevant = sum(1 for course in top_k_pred if course in true_courses)
        recall_scores.append(relevant / len(true_courses))
    return np.mean(recall_scores) if recall_scores else 0.0

# Hit Rate@5
def hit_rate_at_k(y_true, recommendations, k=5):
    hit_rate_scores = []
    for user, recommended_courses in recommendations.items():
        true_courses = y_true.get(user, [])
        if not true_courses:
            continue
        top_k_pred = recommended_courses[:k]
        relevant = any(course in true_courses for course in top_k_pred)
        hit_rate_scores.append(int(relevant))
    return np.mean(hit_rate_scores) if hit_rate_scores else 0.0

# NDCG@5
def ndcg_at_k(y_true, recommendations, k=5):
    dcg_scores = []
    for user, recommended_courses in recommendations.items():
        true_courses = y_true.get(user, [])
        if not true_courses:
            continue
        top_k_pred = recommended_courses[:k]
        dcg_score = sum(1 / np.log2(i + 2) for i, course in enumerate(top_k_pred) if course in true_courses)
        idcg_score = sum(1 / np.log2(i + 2) for i in range(min(k, len(true_courses))))
        dcg_scores.append(dcg_score / idcg_score if idcg_score > 0 else 0.0)
    return np.mean(dcg_scores) if dcg_scores else 0.0

# MAP@5
def map_at_k(y_true, recommendations, k=5):
    ap_scores = []
    for user, recommended_courses in recommendations.items():
        true_courses = y_true.get(user, [])
        if not true_courses:
            continue
        top_k_pred = recommended_courses[:k]
        relevant = [1 if course in true_courses else 0 for course in top_k_pred]
        if sum(relevant) == 0:
            ap_scores.append(0.0)
            continue
        precision_at_i = []
        for i in range(min(k, len(top_k_pred))):
            if relevant[i]:
                precision_at_i.append(sum(relevant[:i+1]) / (i+1))
        ap = sum(precision_at_i) / min(len(true_courses), k)
        ap_scores.append(ap)
    return np.mean(ap_scores) if ap_scores else 0.0

# F1 Score@5
def f1_at_k(y_true, recommendations, k=5):
    f1_scores = []
    for user, recommended_courses in recommendations.items():
        true_courses = y_true.get(user, [])
        if not true_courses:
            continue
        top_k_pred = recommended_courses[:k]
        relevant = sum(1 for course in top_k_pred if course in true_courses)
        precision = relevant / k
        recall = relevant / len(true_courses)
        if precision + recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0.0
        f1_scores.append(f1)
    return np.mean(f1_scores) if f1_scores else 0.0

# Hàm đánh giá mô hình
def evaluate_model(y_true, recommendations, k=5):
    """
    Đánh giá mô hình với Precision@5, Recall@5, Hit Rate@5, NDCG@5, MAP@5, F1 Score@5.

    Parameters:
    y_true: DataFrame hoặc dict chứa các khóa học thực tế trong tập TEST.
    recommendations: dict chứa các khóa học khuyến nghị.
    k: Số khóa học top cần xét (default là 5).

    Returns:
    dict: Các độ đo Precision@5, Recall@5, Hit Rate@5, NDCG@5, MAP@5, F1 Score@5.
    """
    # Chuyển y_true thành dict
    if hasattr(y_true, 'set_index'):
        y_true_dict = {}
        for user, item in y_true.set_index('user')['item'].items():
            y_true_dict[user] = [item] if not isinstance(item, (list, set)) else list(item)
        y_true = y_true_dict
    else:
        y_true = {k: [v] if not isinstance(v, (list, set)) else list(v) for k, v in y_true.items()}

    precision = precision_at_k(y_true, recommendations, k)
    recall = recall_at_k(y_true, recommendations, k)
    hit_rate = hit_rate_at_k(y_true, recommendations, k)
    ndcg = ndcg_at_k(y_true, recommendations, k)
    map_k = map_at_k(y_true, recommendations, k)
    f1 = f1_at_k(y_true, recommendations, k)

    return {
        f"Baseline": f"V{i+1}-DeepFM@{k}",
        f"Precision": round(precision,4),
        f"Recall": round(recall,4),
        f"HitRate": round(hit_rate,4),
        f"NDCG": round(ndcg,4),
        f"MAP": round(map_k,4),
        f"F1-score": round(f1,4)
    }

In [36]:

start_time = time.time()
results1 = recommend_courses(model,test,10)
end_time = time.time()  # Kết thúc đo thời gian

print(f"Thời gian thực hiện: {end_time - start_time:.6f} giây")


Khuyến nghị cho người dùng: 100%|██████████| 24540/24540 [05:25<00:00, 75.38user/s]

Thời gian thực hiện: 325.627659 giây


In [37]:
train[['user','item']]

,user,item
0,U_100090,C_707022
1,U_100090,C_676687
2,U_100090,C_697073
3,U_100090,C_948038
4,U_100090,C_680716
...,...,...
322520,U_997506,C_696673
322521,U_99772,C_697032
322522,U_99772,C_676701
322523,U_99772,C_682485


In [38]:
user_course_train=train[['user','item']]
# 
user_course_test = test[['user','item']]
all_course_ids = pd.concat([train,test],axis=0)['item'].drop_duplicates()

In [39]:
start_time = time.time()
results2 = recommend_courses_with_negative_sampling(model, user_course_train, user_course_test, all_course_ids, num_negatives=100)
end_time = time.time()  # Kết thúc đo thời gian

print(f"Thời gian thực hiện: {end_time - start_time:.6f} giây")
# print(results)

Khuyến nghị với negative sampling: 100%|██████████| 24540/24540 [01:56<00:00, 211.51user/s]

Thời gian thực hiện: 116.992971 giây


In [40]:
eval_5 = evaluate_model(test[["user","item"]], results1,5)
eval_10 = evaluate_model(test[["user","item"]], results1,10)

In [41]:
{k: [eval_5[k], eval_10[k]] for k in eval_5}

{'Baseline': ['V1-DeepFM@5', 'V1-DeepFM@10'],
 'Precision': [0.0128, 0.011],
 'Recall': [0.0639, 0.1105],
 'HitRate': [0.0639, 0.1105],
 'NDCG': [0.0399, 0.0548],
 'MAP': [0.0321, 0.0382],
 'F1-score': [0.0213, 0.0201]}

In [42]:
eval_5 = evaluate_model(test[["user","item"]], results2,5)
eval_10 = evaluate_model(test[["user","item"]], results2,10)
{k: [eval_5[k], eval_10[k]] for k in eval_5}

{'Baseline': ['V1-DeepFM@5', 'V1-DeepFM@10'],
 'Precision': [0.0923, 0.0615],
 'Recall': [0.4615, 0.6147],
 'HitRate': [0.4615, 0.6147],
 'NDCG': [0.3291, 0.3787],
 'MAP': [0.2853, 0.3059],
 'F1-score': [0.1538, 0.1118]}